In [1]:
import os 
# os.chdir(".")
os.chdir("/mnt/petrelfs/jiangshuyang.p/repo/new/MedicalRL")

# cur_dir = os.path.abspath(os.curdir)
# os.chdir(os.path.dirname(os.path.dirname(cur_dir)))
# print(os.path.abspath(os.curdir))
from medicalrl.models import get_model

In [2]:
import torch
torch.set_default_device("cuda")

In [3]:
model_path = "/mnt/petrelfs/jiangshuyang.p/checkpoints/verl_qwen253b_ultra_mcqa80k_base_reinforce_plus_plus"
model = get_model(model_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import json
cache_file_path = "/mnt/petrelfs/jiangshuyang.p/repo/new/MedicalRL/results/medqa_5op/verl_qwen253b_ultra_mcqa80k_base_reinforce_plus_plus_cot/cache_processed.jsonl"
cache_file = [json.loads(line) for line in open(cache_file_path)]


In [5]:
from medicalrl.utils.prompt import get_prepare_func


test_item = cache_file[5]
prepare_func = get_prepare_func(model_path, "base_default")
prompt = prepare_func(test_item['task'])['prompt']

In [ ]:
input_text = prompt.format(prompt=test_item['task']['input'])
# input_text += "\nThe answer is "
logit_bias = 5
outputs = model(input_text, max_new_tokens=512, sample_num=16, temperature=1,
                )


In [7]:
for x in outputs[0]:
    print(x)
    print("-" * 80)

1. The patient presents with progressive worsening chest pain and nausea, which may indicate a myocardial infarction (heart attack) given his history of untreated hypertension and recent cocaine abuse.
2. The patient's diaphoresis and distress further support a suspicion of a cardiac emergency.
3. Among the given options, nitroglycerin is primarily used for unstable angina or a suspected vasospastic chest pain, but it is not the first-line treatment for acute coronary syndrome.
4. Diltiazem, labetalol, and propranolol are beta-blockers that are often used in the treatment of angina and heart failure, but they are not the first-line treatment for myocardial infarction.
5. The first step in the management of a suspected acute coronary syndrome is to stabilize the patient's condition and determine the presence of a myocardial infarction.
6. Therefore, the first step in management should be continuous monitoring and potential immediate interventions such as assessment of cardiac biomarkers

In [29]:
test_input_text = input_text + "1. First, we need to understand the patient's condition. The patient has acute coronary syndrome, which indicates a possible heart attack. The ECG shows ST-segment depressions, and the elevated troponin T level confirms myocardial injury. Therefore, the answer is C"
input_tensor = model.tokenizer(test_input_text, return_tensors="pt")
input_tensor = {k: v.to(model.device) for k, v in input_tensor.items()}
with torch.no_grad():
    logits = model.model(**input_tensor).logits

labels = input_tensor['input_ids'][:, -1:]
logits = logits[:, -2:-1]


In [30]:
import torch.nn.functional as F
def logprobs_from_logits_v2(logits: torch.FloatTensor, labels):
    """
    A memory efficient implementation of logprobs_from_logits
    """
    if logits.dtype in [torch.float32, torch.float64]:
        logits_labels = torch.gather(logits, dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)
        # loop to reduce peak mem consumption
        logsumexp_values = torch.stack([torch.logsumexp(l, dim=-1) for l in logits])
        logprobs_labels = logits_labels - logsumexp_values  # log_softmax(x_i) = x_i - logsumexp(x)
    else:
        # logsumexp approach is unstable with bfloat16, fall back to slightly less efficent approach
        logprobs_labels = []
        for row_logits, row_labels in zip(logits, labels):  # loop to reduce peak mem consumption
            row_logprobs = F.log_softmax(row_logits, dim=-1)
            row_logprobs_labels = row_logprobs.gather(dim=-1, index=row_labels.unsqueeze(-1)).squeeze(-1)
            logprobs_labels.append(row_logprobs_labels)
        logprobs_labels = torch.stack(logprobs_labels)
    return logprobs_labels


In [31]:
# labels = labels.unsqueeze(0)
log_prob = logprobs_from_logits_v2(logits, labels)
print(log_prob)
logits.softmax(-1)[0, 0, labels[0, 0]].item()
# labels

tensor([[-4.1953]], device='cuda:0', dtype=torch.float16)


0.0150909423828125

In [28]:
print(test_input_text)

A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The assistant thinks deeply and outputs the final answer as 'The answer is '. 
User: A 39-year-old man presents to the emergency department because of progressively worsening chest pain and nausea that started at a local bar 30 minutes prior. The pain radiates to the epigastric area. He has a 5-year history of untreated hypertension. He has smoked 1 pack of cigarettes daily for the past 5 years and started abusing cocaine 2 weeks before his emergency room visit. The patient is diaphoretic and in marked distress. What should be the first step in management?
A. Diltiazem
B. Labetalol
C. Nitroglycerin
D. Propranolol
E. Reassurance and continuous monitoring
Assistant:  Therefore, the answer is C
